## Overview ##
---
In this notebook, you will learn how to order a [Planet Basemap](https://developers.planet.com/docs/data/visual-basemaps/) using your [Area of Interest](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-area-of-interest-aoi) (AOI) and a [Quad ID](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-quad-ids-and-deliver-to-cloud). We will place this order via Planet's [Orders API](https://developers.planet.com/apis/orders/) using our Planet Python [SDK](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 

1. Get a Basemap ID using either [Planet Explorer](https://developers.planet.com/docs/apps/explorer/), the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/), or by API request. 
2. Create a JSON order packet with order parameters.
3. Set up a session with the Planet SDK, place the order, and download it.
4. Repeat steps 1-3 using quad IDs. 

### Import Planet and Related Packages

---

Make sure you have Planet's Python SDK properly downloaded. You can find out more about this [here](https://developers.planet.com/docs/pythonclient/). Find your [API key](https://developers.planet.com/quickstart/apis/).

Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [ ]:
import planet
import os
import json
import requests
from requests.auth import HTTPBasicAuth

In [ ]:
PL_API_KEY = os.environ.get('PL_API_KEY')
auth = planet.Auth.from_key(PL_API_KEY)

### Get your basemap names

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
auth = HTTPBasicAuth(PL_API_KEY, '')
basemapServiceResponse = requests.get(url=BASEMAP_API_URL, auth=auth)

In [ ]:
basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
mosaics = basemaps['mosaics']
names = [feature['name'] for feature in mosaics]
print(names)

Iterate through all your available basemaps and extract all the global monthly from 2020.

In [ ]:
global_monthly = []
cont = True
while cont:
    for name in names:
        #Here I am filtering for the global monthly basemap
        if "2020" in name and "global_monthly" in name:
            global_monthly.append(name)
    next_url = basemaps['_links']['_next']
    basemapServiceResponse = requests.get(url=next_url, auth=auth)
    basemaps = json.loads(basemapServiceResponse.text)
    mosaics = basemaps['mosaics']
    names = [feature['name'] for feature in mosaics]
    if "_next" not in basemaps["_links"]:
        cont = False

In [ ]:
global_monthly

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. You can see examples of using tools [here](https://developers.planet.com/apis/orders/tools/). Make sure to replace the mosaic name and coordinates with your specifications.

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

In [ ]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": global_monthly[0],
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "tools": [{"clip": {}}]
}

### Create a session with SDK and poll for success

Here, we are creating an order using the SDK. Then we are waiting for the order to be delievered to our GCP storage bucket. Note that this can take some time.

In [ ]:
async def create_and_deliver_order(order_params, client):
    '''Create an order and wait for it to delivered

    Parameters:
        order_params: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)
        # if we get here that means the order completed. Yay! Download the files.
    await client.download_order(order['id'])

In [ ]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await create_and_deliver_order(order_params, client)

## Creating a streaming link for scenes

This quick demo shows you how to use the tiles API to create WMTS links to stream into you GIS

In [ ]:
tile_url = 'https://tiles0.planet.com/data/v1/layers'
#Copy the list of scene IDs from the API dialog into the ids JSON
data = {
  "ids": [
    "PSScene:20230813_184925_91_2475"
  ]
}
api_key = os.getenv('PL_API_KEY')

In [ ]:
res = requests.post(tile_url, auth=(api_key, ""), data=data)
print(res)
print(res.json())
name = res.json()["name"]
url = "{}/wmts/{}?api_key={}".format(tile_url,name, api_key)

# This URL includes your API key, be careful where you share it!
print(url)